In [2]:
import pandas as pd
import numpy as np
import csv
from tqdm.notebook import tqdm
from multiprocessing import Process

In [3]:
def distance_from_lonlat(p1,p2):
    # https://github.com/openlayers/openlayers/blob/release-2.12/lib/OpenLayers/Util.js#L750
    #VincentyConstants
    a = 6378137
    b = 6356752.3142
    f = 1/298.257223563
    
    p1_lon = (np.pi/180) * p1[0]
    p1_lat = (np.pi/180) * p1[1]
    p2_lon = (np.pi/180) * p2[0]
    p2_lat = (np.pi/180) * p2[1]
    
    L = p2_lon - p1_lon
    U1 = np.arctan((1-f) * np.tan(p1_lat))
    U2 = np.arctan((1-f) * np.tan(p2_lat))
    cosU1 = np.cos(U1)
    cosU2 = np.cos(U2)
    sinU1 = np.sin(U1)
    sinU2 = np.sin(U2)
    
    
    lambda_ = L
    lambdaP = 2*np.pi
    iterLimit = 30
    
    while(np.abs(lambda_ - lambdaP) > 1e-13):
        sinSigma = np.sqrt((cosU2*np.sin(lambda_))**2 + (cosU1*sinU2- sinU1*cosU2*np.cos(lambda_))**2)
        if (sinSigma == 0):
            return 0
        cosSigma = sinU1*sinU2 + cosU1*cosU2*np.cos(lambda_)
        sigma = np.arctan2(sinSigma, cosSigma)
        alpha = np.arcsin(cosU1*cosU2*np.sin(lambda_)/sinSigma)
        cosalpha = np.cos(alpha)
        sinalpha = np.sin(alpha)
        CSM   = cosSigma - 2*sinU1*sinU2/(cosalpha**2)
        C     = (f/16)*cosalpha**2*(4+f*(4-3*(cosalpha**2)))
        lamdaP = lambda_
        lambda_ = L + (1-C)*f*sinalpha*(sigma + C*sinSigma*(CSM + C*cosSigma*(-1 +2*CSM**2)))
        iterLimit -= 1
        if iterLimit <= 0:
                        break
    uSq = cosalpha*(a**2 - b **2)/(b**2)
    A = 1 + uSq/16384*(4096+uSq*(-768+uSq*(320-175*uSq)))
    B = uSq/1024 * (256+uSq*(-128+uSq*(74-47*uSq)))
    deltaSigma = B*sinSigma*(CSM+B/4*(cosSigma*(-1+2*CSM**2)-
    B/6*CSM*(-3+4*sinSigma*sinSigma)*(-3+4*CSM**2)))
    s = b*A*(sigma-deltaSigma)
    return s

In [4]:
bus_lat_list = []
bus_lon_list = []
with open('bus_stop_coordinates.csv') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        bus_lat_list.append(float(row[0]))
        bus_lon_list.append(float(row[1]))

zipped_bus = list(zip(bus_lon_list,bus_lat_list))


df = pd.read_csv("Data_Lv3_UMDOTS_Escooters.csv")


lon_list_start = list(map(float,df['START LONG'][:]))
lat_list_start = list(map(float,df['START LAT'][:])) 
zipped_start = list(zip(lon_list_start,lat_list_start))

lon_list_end = list(map(float,df['END LONG'][:]))
lat_list_end = list(map(float,df['END LAT'][:]))
zipped_end = list(zip(lon_list_end,lat_list_end))

In [5]:
radius = 50

thread_1_list = []
thread_2_list = []
thread_3_list = []
thread_4_list = []

def thread_1():
    for i in tqdm(range(6)):
        for j in tqdm(range(len(zipped_bus)),leave=False):
            for k in range(len(zipped_start)):
                start = zipped_start[k]
                if (distance_from_lonlat(zipped_bus[i],start) < radius):
                    end   = zipped_end[k] 
                    if((distance_from_lonlat(zipped_bus[j],end) < radius)):
                        thread_1_list.append([i,j,df['MINUTES'][k]])
    
def thread_2():
    for i in tqdm(range(6,13)):
        for j in tqdm(range(len(zipped_bus)),leave=False):
            for k in range(len(zipped_start)):
                start = zipped_start[k]
                if (distance_from_lonlat(zipped_bus[i],start) < radius):
                    end   = zipped_end[k] 
                    if((distance_from_lonlat(zipped_bus[j],end) < radius)):
                        thread_2_list.append([i,j,df['MINUTES'][k]]) 
                    
def thread_3():
    for i in tqdm(range(13,18)):
        for j in tqdm(range(len(zipped_bus)),leave=False):
            for k in range(len(zipped_start)):
                start = zipped_start[k]
                if (distance_from_lonlat(zipped_bus[i],start) < radius):
                    end   = zipped_end[k] 
                    if((distance_from_lonlat(zipped_bus[j],end) < radius)):
                        thread_3_list.append([i,j,df['MINUTES'][k]])

def thread_4():
    for i in tqdm(range(18,22)):
        for j in tqdm(range(len(zipped_bus)),leave=False):
            for k in range(len(zipped_start)):
                start = zipped_start[k]
                if (distance_from_lonlat(zipped_bus[i],start) < radius):
                    end   = zipped_end[k] 
                    if((distance_from_lonlat(zipped_bus[j],end) < radius)):
                        thread_4_list.append([i,j,df['MINUTES'][k]]) 
    
p1 = Process(target=thread_1)
p2 = Process(target=thread_2)
p3 = Process(target=thread_3)
p4 = Process(target=thread_4)



Process Process-4:
Traceback (most recent call last):
  File "/usr/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-5-c1d08c2af0d0>", line 43, in thread_4
    if (distance_from_lonlat(zipped_bus[i],start) < radius):
IndexError: list index out of range


In [22]:
with open('rides_between_bus_stops.csv','w') as csvfile:
    writer = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(['Start_idx', 'End_idx', 'Duration'])
    for row in thread_1_list:
        writer.writerow(row)
    for row in thread_2_list:
        writer.writerow(row)
    for row in thread_3_list:
        writer.writerow(row)
    for row in thread_4_list:
        writer.writerow(row)

0         1
1        30
2         8
3        47
4         4
         ..
40320     5
40321     5
40322     5
40323    13
40324    35
Name: MINUTES, Length: 40325, dtype: int64

In [7]:
thread_3_list

[]